In [ ]:
!pip install faster-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 14.2 MB/s eta 0:00:00
  Created wheel for faster-whisper: filename=faster_whisper-0.10.0-py3-none-any.whl size=1539727 sha256=f8784eae9a6a4cda169c951c82cdf09b0c5f3215adecfc697876c7a78f22149e
  Stored in directory: /root/.cache/pip/wheels/b3/4e/9a/bd36d2645cb73f909a3a65a2e317fec5c6a79c8121ab9eb42f
Successfully built faster-whisper


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('cmudict')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize #idk

from collections import Counter # a list counter into a dict (hash)

import regex as re

import numpy as np

In [ ]:
from nltk.corpus import cmudict
d = cmudict.dict()

In [ ]:
from faster_whisper import WhisperModel

In [ ]:
import librosa
import numpy as np
import glob
import os
import regex as re
import json
import time

In [ ]:
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
!sudo dpkg -i libssl1.1_1.1.0g-2ubuntu4_amd64.deb

--2023-11-30 05:21:20--  http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
Resolving archive.ubuntu.com (archive.ubuntu.com)... 91.189.91.81, 91.189.91.83, 91.189.91.82, ...
Connecting to archive.ubuntu.com (archive.ubuntu.com)|91.189.91.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1128092 (1.1M) [application/vnd.debian.binary-package]
Saving to: ‘libssl1.1_1.1.0g-2ubuntu4_amd64.deb’

libssl1.1_1.1.0g-2u 100%[===================>]   1.08M  4.94MB/s    in 0.2s    

2023-11-30 05:21:20 (4.94 MB/s) - ‘libssl1.1_1.1.0g-2ubuntu4_amd64.deb’ saved [1128092/1128092]

Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack libssl1.1_1.1.0g-2ubuntu4_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.0g-2ubuntu4) ...
Setting up libssl1.1:amd64 (1.1.0g-2ubuntu4) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-lik

In [ ]:
!pip install azure-cognitiveservices-speech

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.4 MB/s eta 0:00:00


In [ ]:
!pip install pydub

In [ ]:
import azure.cognitiveservices.speech as speechsdk
from pydub import AudioSegment

In [ ]:
speech_key, service_region = "<your_azure_key>", "<your_azure_region>"

In [ ]:
# Run on GPU with FP16
model = WhisperModel("large-v2", device="cuda", compute_type="float16")

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

In [ ]:
def syllables_aux(word: str) -> int:
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if(len(word)>0):
        if word[0] in vowels:
            count +=1
        for index in range(1,len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count +=1
        if word.endswith('e'):
            count -= 1
        if word.endswith('le'):
            count += 1
        if count == 0:
            count += 1
    return count

In [ ]:
def count_word_syllables(word: str):
    try:
        # return the pronunciation with the minimum number of syllables
        return min([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    except KeyError:
        return syllables_aux(word)

In [ ]:
def get_audio_duration(file_path: str):
   audio_data, sample_rate = librosa.load(file_path)
   duration = librosa.get_duration(y=audio_data, sr=sample_rate)
   return duration

In [ ]:
def clean_word(word: str) -> str:
    lower = word.lower()
    new_str = ""
    for c in lower:
        if c.isalpha():
            new_str += c
    return new_str

In [ ]:
def is_filler_word(word: str) -> str:
    return re.fullmatch(r"u+m+|e+m+|e+r+|e+h+|u+h+", word) is not None

In [ ]:
def pronunciation_assessment_continuous_from_file(file_name: str, reference_text: str, use_recognition_once: bool):
    """Performs continuous pronunciation assessment asynchronously with input from an audio file.
        See more information at https://aka.ms/csspeech/pa"""

    # Create wav file from mp3
    sound = AudioSegment.from_mp3(file_name)
    wav_file_name = file_name[:-3] + "wav"
    sound.export(wav_file_name, format="wav")

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=wav_file_name)

    pronunciation_config = speechsdk.PronunciationAssessmentConfig(reference_text=reference_text, enable_miscue=True)
    # pronunciation_config.nbest_phoneme_count = 2

    # Creates a speech recognizer using a file as audio input.
    language = 'en-US'
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language=language, audio_config=audio_config)

    # apply pronunciation assessment config to speech recognizer
    pronunciation_config.apply_to(speech_recognizer)

    done = False
    assessments = []

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        # print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    def recognized(evt: speechsdk.SpeechRecognitionEventArgs):
        nonlocal assessments
        pronunciation_assessment_result_json = evt.result.properties.get(
            speechsdk.PropertyId.SpeechServiceResponse_JsonResult)
        assessments.append(pronunciation_assessment_result_json)

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognized.connect(recognized)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous pronunciation assessment
    if use_recognition_once:
        speech_recognizer.recognize_once()
    else:
        speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)
    return assessments

In [ ]:
def get_errors_from_assessment(result: str) -> int:
    n_errors = 0
    for obj in result:
        data = json.loads(obj)
        best1 = data["NBest"][0]
        for word in best1["Words"]:
            error_type = word["PronunciationAssessment"]["ErrorType"]
            if error_type != "None":
                print(error_type)
                n_errors += 1
    return n_errors

In [ ]:
def mfcc_feature_extraction(file_name):
    X , sample_rate = librosa.load(file_name, sr=None) #Can also load file using librosa
    if X.ndim > 1:
        X = X[:,0]
    X = X.T

    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=20).T, axis=0) #Returns N_mel coefs
    rmse = np.mean(librosa.feature.rms(y=X).T, axis=0) #RMS Energy for each Frame (Stanford's). Returns 1 value
    spectral_flux = np.mean(librosa.onset.onset_strength(y=X, sr=sample_rate).T, axis=0) #Spectral Flux (Stanford's). Returns 1 Value
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=X).T, axis=0) #Returns 1 value

    ##Return computed audio features
    return mfccs, rmse, spectral_flux, zcr

In [ ]:
def features_yu(audio_file: str, debug: bool = False) -> np.array:
    """
    (1) articulation rate = number of syllables, including disfluencies, divided by to-
    tal duration of speech apart from all (silent and filled) pauses longer than
    0.25 seconds;
    (2) speech rate = as for articulation rate, but including all pauses in the total
    speech duration;
    (3) effective speech rate = as for speech rate, but excluding disfluencies from the
    syllable count;
    (4) number of silent pauses above 0.25 seconds in duration;
    (5) mean length of silent pauses longer than 0. 25 seconds;
    (6) number of filled pauses (uh, er, mm, etc.);
    (7) mean length of all filled pauses;
    (8) number of pauses = sum of (4) and (6);
    (9) mean length of pauses = mean of (5) and (7), weighted by their respective
    frequencies (items 4 and 6);
    (10) number of other disfluencies (repetitions, restarts, false starts, corrections);
    (11) mean length of fluent runs = mean number of syllables produced between
    silent pauses longer than 0.25 seconds;
    (12) phonation/time ratio, calculated on the basis of items 4 and 6, as a percentage
    of overall speech time = (total duration of speech without pauses, divided by
    total duration of speech including pauses) × 100
    """
    try:
        results, _ = model.transcribe(audio_file,
                                language="en", word_timestamps=True, beam_size=5)
        audio_duration = get_audio_duration(audio_file)
        silent_pauses = []
        filled_pauses = []
        duration_of_speech_excluding_pauses = 0
        fluent_runs_syllables = []
        last_segment = None
        final_transcription = ""
        for segment in results:
            if debug: print("New segment")
            final_transcription += segment.text + " "
            if last_segment is not None:
                silent_pause = segment.words[0].start - last_segment.words[-1].end
                if silent_pause > 0.25:
                    silent_pauses.append(silent_pause)
            for i in range(len(segment.words)):
                word = segment.words[i]
                if debug: print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))
                # Compute silent pauses
                if i <= len(segment.words) - 2:
                    next = segment.words[i+1]
                    silent_pause = next.start - word.end
                    if silent_pause > 0.25:
                        silent_pauses.append(silent_pause)
                clean = clean_word(word.word)
                # Compute filled pauses
                if is_filler_word(clean):
                    filled_pauses.append(word.end - word.start)
                else:
                    fluent_runs_syllables.append(count_word_syllables(clean))
                    duration_of_speech_excluding_pauses += word.end - word.start
            last_segment = segment
        if debug:
            print("Silent pauses:", silent_pauses)
            print("Filled pauses:", filled_pauses)
            print("Fluent runs (syllables):", fluent_runs_syllables)
            print("Starting assessment with transcription:", final_transcription)
        assessment = pronunciation_assessment_continuous_from_file(audio_file,
                                                                reference_text=final_transcription, use_recognition_once=True)
        if debug: print("Finished assessment:", assessment)
        articulation_rate = ((sum(fluent_runs_syllables) + len(filled_pauses)) / duration_of_speech_excluding_pauses) if duration_of_speech_excluding_pauses > 0 else 0
        speech_rate = ((sum(fluent_runs_syllables) + len(filled_pauses)) / (duration_of_speech_excluding_pauses + sum(filled_pauses) + sum(silent_pauses))) if (duration_of_speech_excluding_pauses + sum(filled_pauses) + sum(silent_pauses)) > 0 else 0
        effective_speech_rate = (sum(fluent_runs_syllables) / duration_of_speech_excluding_pauses) if duration_of_speech_excluding_pauses > 0 else 0
        number_of_silent_pauses = len(silent_pauses)
        mean_length_of_silent_pauses = (sum(silent_pauses) / number_of_silent_pauses) if number_of_silent_pauses > 0 else 0
        number_of_filled_pauses = len(filled_pauses)
        mean_length_of_filled_pauses = (sum(filled_pauses) / number_of_filled_pauses) if number_of_filled_pauses > 0 else 0
        number_of_pauses = number_of_silent_pauses + number_of_filled_pauses
        mean_length_of_pauses = ((mean_length_of_silent_pauses * number_of_silent_pauses + mean_length_of_filled_pauses * number_of_filled_pauses) / number_of_pauses) if number_of_pauses > 0 else 0
        number_of_other_disfluencies = get_errors_from_assessment(assessment)
        mean_length_of_fluent_runs = (sum(fluent_runs_syllables) / len(fluent_runs_syllables)) if len(fluent_runs_syllables) > 0 else 0
        phonation_ratio = (duration_of_speech_excluding_pauses / (duration_of_speech_excluding_pauses + sum(filled_pauses) + sum(silent_pauses))) if (duration_of_speech_excluding_pauses + sum(filled_pauses) + sum(silent_pauses)) > 0 else 0
        return np.array([
            articulation_rate,
            speech_rate,
            effective_speech_rate,
            number_of_silent_pauses,
            mean_length_of_silent_pauses,
            number_of_filled_pauses,
            mean_length_of_filled_pauses,
            number_of_pauses,
            mean_length_of_pauses,
            number_of_other_disfluencies,
            mean_length_of_fluent_runs,
            phonation_ratio
        ])
    except Exception as e:
        print(e)
        return np.zeros(12)

In [ ]:
def features_prosodic(audio_file: str, debug: bool = False) -> np.array:
    """
    (1) Average number of filled-pauses per sec. (AvgFP)
    (2) Average duration of a filled-pause       (DurFP) (Equals to 7th Yu's feature)
    (3) Average distance between filled-pauses   (DistFP)
    (4) Length of the longest filled pause       (MaxFP)
    (5) Fraction of silence                      (FracSIL)
    (6) Average duration of contiguous silence   (DurSIL) (Equals to 5th Yu's feature)
    (7) Average duration of contiguous speech    (DurSP)
    (8) Average distance between silences        (DistSIL)
    """
    results, _ = model.transcribe(audio_file,
                              language="en", word_timestamps=True, beam_size=5)
    audio_duration = get_audio_duration(audio_file)
    silent_pauses = []
    filled_pauses = []
    duration_of_speech_excluding_pauses = 0
    fluent_runs_syllables = []
    last_segment = None
    final_transcription = ""
    for segment in results:
        if debug: print("New segment")
        final_transcription += segment.text + " "
        if last_segment is not None:
            silent_pause = segment.words[0].start - last_segment.words[-1].end
            if silent_pause > 0.25:
                silent_pauses.append(silent_pause)
        for i in range(len(segment.words)):
            word = segment.words[i]
            if debug: print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))
            # Compute silent pauses
            if i <= len(segment.words) - 2:
                next = segment.words[i+1]
                silent_pause = next.start - word.end
                if silent_pause > 0.25:
                    silent_pauses.append(silent_pause)
            clean = clean_word(word.word)
            # Compute filled pauses
            if is_filler_word(clean):
                filled_pauses.append(word.end - word.start)
            else:
                fluent_runs_syllables.append(count_word_syllables(clean))
                duration_of_speech_excluding_pauses += word.end - word.start
        last_segment = segment
    if debug:
        print("Silent pauses:", silent_pauses)
        print("Filled pauses:", filled_pauses)
        print("Fluent runs (syllables):", fluent_runs_syllables)


    def get_distances_between_frames(frames) -> list:
      if len(frames) < 2: return []
      distances = []
      for i in range(1, len(frames)):
        dist = abs(frames[i-1] - frames[i])
        distances.append(dist)
      return distances

    # Primary features
    number_of_filled_pauses = len(filled_pauses)
    number_of_silent_pauses = len(silent_pauses)

    # Prosodic Features
    # 1) Average number of filled-pauses per sec. (AvgFP)
    mean_number_of_filled_pauses_per_sec = number_of_filled_pauses / audio_duration

    # 2) Average duration of a filled-pause       (DurFP) (Equals to 7th Yu's feature)
    mean_length_of_filled_pauses = sum(filled_pauses) / number_of_filled_pauses if number_of_filled_pauses else 0

    # 3) Average distance between filled-pauses   (DistFP)
    distances_between_fp = get_distances_between_frames(filled_pauses)
    mean_distance_between_fp = sum(distances_between_fp)/len(distances_between_fp) if len(filled_pauses) >= 2 else 0

    # 4) Length of the longest filled pause       (MaxFP)
    length_longest_fp = max(filled_pauses) if len(filled_pauses) > 0 else 0

    # 5) Fraction of silence                      (FracSIL)
    fraction_of_silence = sum(silent_pauses) / audio_duration

    # 6) Average duration of contiguous silence   (DurSIL) (Equals to 5th Yu's feature)
    mean_length_of_silent_pauses = sum(silent_pauses) / number_of_silent_pauses if number_of_silent_pauses else 0

    # 7) Average duration of contiguous speech    (DurSP)
    mean_duration_of_contiguous_speech = duration_of_speech_excluding_pauses / len(fluent_runs_syllables) if len(fluent_runs_syllables) != 0 else 0

    # 8) Average distance between silences        (DistSIL)
    distances_between_silences = get_distances_between_frames(silent_pauses)
    mean_distance_between_silences = sum(distances_between_silences)/len(distances_between_silences) if len(silent_pauses) >= 2 else 0


    return np.array([
        mean_number_of_filled_pauses_per_sec,
        mean_length_of_filled_pauses,
        mean_distance_between_fp,
        length_longest_fp,
        fraction_of_silence,
        mean_length_of_silent_pauses,
        mean_duration_of_contiguous_speech,
        mean_distance_between_silences
    ])

Lexical Features

In [ ]:
ps = PorterStemmer()

In [ ]:
stop_words = set(stopwords.words('english'))
discourse_markers_no_proccessed = set({'and','also','moreover','too','furthermore','additionally','such','because','so',
                     'therefore','thus','consequently','hence','similarly','likewise','like','equally','first','second','third',
                     'next','meanwhile','after','then','subsequently','especially','significantly','indeed','notably','but',
                     'however','although','unless','except','apart','if','whereas','instead','alternatively','otherwise','unlike','conversely'})
discourse_markers = set({})
for connector in discourse_markers_no_proccessed:
  connector = ps.stem(connector)
  if connector not in stop_words:
    discourse_markers.add(connector)

discourse_markers

{'addit',
 'also',
 'altern',
 'although',
 'apart',
 'becaus',
 'consequ',
 'convers',
 'equal',
 'especi',
 'except',
 'first',
 'furthermor',
 'henc',
 'howev',
 'inde',
 'instead',
 'like',
 'likewis',
 'meanwhil',
 'moreov',
 'next',
 'notabl',
 'otherwis',
 'second',
 'significantli',
 'similarli',
 'subsequ',
 'therefor',
 'third',
 'thu',
 'unless',
 'unlik',
 'wherea'}

In [ ]:
def is_filler_word(word: str) -> str:
    return re.fullmatch(r"u+m+|e+m+|e+r+|e+h+|u+h+", word) is not None

In [ ]:
def lexical_features(audio_file: str, debug: bool = False) -> np.array:
  """
  (1) Count of most frequent words                : FreqW
  (2) Total words                                 : TW
  (3) Total unique words                          : TUW
  (4) Count of filled-pauses                      : CFP
  (5) Count of dictionary words                   : Cwrd
  (6) Total repeated 'similar' trigrams           : RepTri
  (7) No. of closely occurring unigrams           : ClUni
  (8) No. of closely occurring similar trigrams   : ClTri
  """
  results, _ = model.transcribe(audio_file, language="en", word_timestamps=True, beam_size=5, initial_prompt = "Transcribe me all kind of filled pauses such as um, em, er, erm, eh, uh, uhm, hm, mm.")
  words_stemmed: list(str) = []
  for segment in results:
    for word in segment.words:
        element: str = fromWordtoString(word.word)
        if element not in stop_words:
          words_stemmed.append(ps.stem(element))
        # print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))
  dictFreqW = {}
  for word in words_stemmed:
    if word in discourse_markers:
      if word not in dictFreqW:
        dictFreqW[word] = 1
      else:
        dictFreqW[word] += 1
  # captures the speaker’s favourite discourse marker
  FreqW = 0
  for key in dictFreqW:
    FreqW = max(FreqW, dictFreqW[key])
  # is the count of total words in the transcript
  TW = len(words_stemmed)
  # is a good indicator of the speaker’s vocabulary
  TUW = 0
  dictTUW = dict(Counter(words_stemmed))
  TUW = len(dictTUW)
  # which counts the total number of filled pauses in the transcript
  CFP = 0
  for word in words_stemmed:
    if is_filler_word(word):
      CFP += 1
  # is the difference of (2) and (4)
  Cwrd = TW - CFP
  # two trigrams are called ’similar’ if they differ by at most a single word deletion, insertion of substitution
  RepTri = 0
  sz = len(words_stemmed)
  already_visited = []
  for i in range(sz):
      PIECE = 0
      for j in range(i+1,sz):

          if words_stemmed[i] not in already_visited and (words_stemmed[i] == words_stemmed[j]):
              listA = []
              listB = []
              for k in range(4):
                  if i+k < sz:
                      listA.append(words_stemmed[i+k])
                  if j+k < sz:
                      listB.append(words_stemmed[j+k])


              if(len(listA) <= 2 or (len(listB) <= 2)):
                  continue
              cont = 0

              s_iter = 0
              for m in range(0,len(listA)):
                  for s in range(s_iter,len(listB)):
                      if (listA[m] == listB[s]):
                          cont += 1
                          s_iter += 1

              if(cont >= 3):
                  PIECE += 1
                  if debug: print("Check ", end=" ")


              if debug: print(listA, listB)
      already_visited.append(words_stemmed[i])
      if(PIECE): RepTri += PIECE + 1
  # captures the number of instances of one-word repetition disfluencies with at the most one-word-long edit phrases while feature
  ClUni = 0
  # Assuring its at most one-word long
  dictSet = set()
  for i in range(len(words_stemmed)-1):
    if (words_stemmed[i] == words_stemmed[i+1]):
      ClUni += 1
      dictSet.add(words_stemmed[i])

  ClUni += len(dictSet)
  # captures the number of repetition disfluencies where the reparandum is three-word long and the edit-phrase is at the most two-word long
  # Currently is difficult to analyze how to detect reparandum
  ClTri = 0
  if debug:
    print(f"Count of most frequent words (FreqW): {FreqW}")
    print(f"Total words (TW): {TW}")
    print(f"Total unique words (TUW): {TUW}")
    print(f"Count of filled-pauses (CFP): {CFP}")
    print(f"Count of dictionary words (Cwrd): {Cwrd}")
    print(f"Total repeated 'similar' trigrams (RepTri): {RepTri}")
    print(f"No. of closely occurring unigrams (ClUni): {ClUni}")
    print(f"No. of closely occurring similar trigrams (ClTri): {ClTri}")
    print(words_stemmed)
    print(results)
  return np.array([
      FreqW,
      TW,
      TUW,
      CFP,
      Cwrd,
      RepTri,
      ClUni
      # ClTri
  ])


In [ ]:
def fromWordtoString(word: str) -> str:
  ans = ""
  pattern = r'[aA-zZ|\']'

  for letter in word:
    if re.match(pattern,letter):
      ans = ans + letter

  return ans.lower()

# Extract all features and save to file

In [ ]:
PATH = "<path_to_dataset>"

In [ ]:
audio_subdirectories = os.listdir(PATH)
audio_subdirectories.sort()
print('Audio Subdirs: ', audio_subdirectories)

In [ ]:
def parse_audio_files_mfcc(parent_dir, sub_dirs, file_ext='*.mp3'): # Audio Format
    n_mfccs = 20 # This variable is tunneable with each run
    number_of_features = 3 + n_mfccs
    #number_of_features = 154 + n_mfccs # 154 are the total values returned by rest of computed features
    features, labels = np.empty((0,number_of_features)), np.empty(0)
    filenames = np.empty(0, dtype=object)
    ##Extract features for each audio file
    for label, sub_dir in enumerate(sub_dirs): ##The enumerate() function adds a counter to an iterable.
        for file_name in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)): ##parent is data, sub_dirs are the classes
            print("Actual File Name: ", file_name)
            try:
                mfccs, rmse, spectral_flux, zcr = feature_extraction(file_name)
            except Exception as e:
                print("[Error] there was an error in feature extraction. %s" % (e))
                continue
            extracted_features = np.hstack([mfccs, rmse, spectral_flux, zcr])
            features = np.vstack([features, extracted_features]) #Stack arrays in sequence vertically (row wise).
            labels = np.append(labels, label)
            filenames = np.append(filenames, file_name)
        print("Extracted features from %s, done" % (sub_dir))
    return np.array(features), np.array(labels, dtype = int), filenames ## arrays with features and

In [ ]:
def parse_audio_files_yu(parent_dir, sub_dirs, file_ext='*'): # Audio Format
    features = np.empty((0, N_FEATURES_YU))
    labels = np.empty(0, dtype=int)
    filenames = np.empty(0, dtype=object)
    for label, sub_dir in enumerate(sub_dirs):
        for file_name in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            print("Actual File Name: ", file_name)
            try:
                extracted_features = features_yu(file_name)
            except Exception as e:
                print("[Error] there was an error in feature extraction. %s" % (e))
                continue
            print("Features: ", extracted_features)
            features = np.vstack([features, extracted_features])
            labels = np.append(labels, label)
            filenames = np.append(filenames, file_name)
        print("Extracted features from %s, done" % (sub_dir))
    return features, labels, filenames

In [ ]:
def parse_audio_files_lexical_prosodic(parent_dir, sub_dirs, file_ext='*.mp3'): # Audio Format
    features = np.empty((0, 8+7))
    labels = np.empty(0, dtype=int)
    filenames = np.empty(0, dtype=object)
    for label, sub_dir in enumerate(sub_dirs):
        i = 0
        for file_name in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            if i % 100 == 0:
              print("Actual File Name: ", file_name)
            try:
                extracted_prosodic_features = features_prosodic(file_name, 0)
                extracted_lexical_features = lexical_features(file_name, 0)
                extracted_features = np.concatenate([extracted_prosodic_features, extracted_lexical_features])
            except Exception as e:
                print("[Error] there was an error in feature extraction. %s" % (e))
                continue
            if i % 100 == 0:
              print("Features: ", extracted_features)
            i += 1
            features = np.vstack([features, extracted_features])
            labels = np.append(labels, label)
            filenames = np.append(filenames, file_name)
        print("Extracted features from %s, done" % (sub_dir))
    return features, labels, filenames

In [ ]:
features, labels, filenames = parse_audio_files_mfcc(PATH, audio_subdirectories) #(parent dir,sub dirs)
np.save('mfcc-features.npy', features)
np.save('mfcc-labels.npy', labels)
np.save('mfcc-filenames.npy', filenames)

In [ ]:
features, labels, filenames = parse_audio_files_yu(PATH, audio_subdirectories) #(parent dir,sub dirs)
np.save('yu-features.npy', features)
np.save('yu-labels.npy', labels)
np.save('yu-filenames.npy', filenames)

In [ ]:
features, labels, filenames = parse_audio_files_lexical_prosodic(PATH, audio_subdirectories) #(parent dir,sub dirs)
np.save('lexical-prosodic-features.npy', features)
np.save('lexical-prosodic-labels.npy', labels)
np.save('lexical-prosodic-filenames.npy', filenames)